In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

In [3]:
eligibility = client.to_pandas("""
    select
        p.project_id,
        p.project_name,
        p.display_name,
        e.transaction_count,
        e.active_days,
        e.active_addresses_count,
        e.gas_fees,
        e.is_eligible,
        e.has_defillama_adapter,
        e.has_bundle_bear
    from oso.int_superchain_s7_onchain_builder_eligibility e
    join oso.projects_v1 p on e.project_id = p.project_id
    join oso.projects_by_collection_v1 pbc on p.project_id = pbc.project_id
    where pbc.collection_name = '8-1'
    order by e.transaction_count desc
""")
eligibility.head()

,project_id,project_name,display_name,transaction_count,active_days,active_addresses_count,gas_fees,is_eligible,has_defillama_adapter,has_bundle_bear
0,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e4...,Aerodrome Finance,296739144,180,1404902,9728.068303,True,True,False
1,Jr4krZ4lp1dMiTRDwmF2R9Hw4mF3rsukcBCM4pnF/fc=,0x9d9162fac637991d2a56da8fbe03dba24c87eb82d47c...,Uniswap on Superchain (Oku),98316074,180,26568519,4675.550456,True,True,False
2,5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be...,Velodrome Finance,57137344,181,518085,265.323322,True,True,False
3,HgZEmwMqbOcnLVW065exx9nhiBr6eoEA2any9TRsGfc=,0x780cf3d5aa4e3c94b8a4157f4b06a3f92ebcc2081358...,Virtuals Protocol,56318779,180,932912,1691.233509,True,False,False
4,MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,0xb98778ca9ff41446e2bc304f7b5d27f0fa7c2bcd11df...,Account Abstraction - ERC-4337,22635189,180,2470,1496.828921,True,False,True


In [47]:
gm_eligibility = client.to_pandas("""

WITH events AS (
  SELECT *
  FROM oso.int_superchain_events_by_project
  WHERE    
    "time" BETWEEN DATE('2024-10-01') AND DATE('2025-02-28')
    AND event_type IN ('CONTRACT_INVOCATION', 'CONTRACT_INTERNAL_INVOCATION')
    AND project_id = 'klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs='
),

builder_metrics AS (
  SELECT
    project_id,
    TRUE as applied_to_round,
    COUNT(DISTINCT transaction_hash) AS transaction_count,
    COUNT(DISTINCT from_artifact_id) AS active_addresses_count,
    COUNT(DISTINCT DATE_TRUNC('DAY', events.time)) AS active_days,
    SUM(events.gas_fee) AS gas_fees
  FROM events
  GROUP BY project_id
),

project_eligibility AS (
  SELECT
    project_id,
    (
      CAST(transaction_count >= 1000 AS INTEGER) +
      CAST(active_days >= 10 AS INTEGER) +
      CAST(active_addresses_count >= 420 AS INTEGER) +
      CAST(gas_fees >= 0.0 AS INTEGER)
      >= 3
    ) AS meets_all_criteria
  FROM builder_metrics
),

artifacts AS (
  SELECT DISTINCT
    project_id,
    'DEFILLAMA_ADAPTER' AS artifact_type
  FROM oso.artifacts_by_project_v1
  WHERE artifact_source = 'DEFILLAMA'

  UNION ALL

  SELECT DISTINCT
    p2p.external_project_id AS project_id,
    'BUNDLE_BEAR' AS artifact_type
  FROM oso.int_projects_to_projects AS p2p
  JOIN oso.artifacts_by_project_v1 AS abp
    ON p2p.artifact_id = abp.artifact_id
  WHERE
    p2p.ossd_project_name IN (
      SELECT DISTINCT ossd_name
      FROM oso.int_4337_address_labels
    )
    AND p2p.external_project_source = 'OP_ATLAS'
    AND abp.artifact_source = 'GITHUB'
),

artifact_flags AS (
  SELECT
    project_id,
    MAX(CASE WHEN artifact_type = 'DEFILLAMA_ADAPTER' THEN TRUE ELSE FALSE END) AS has_defillama_adapter,
    MAX(CASE WHEN artifact_type = 'BUNDLE_BEAR' THEN TRUE ELSE FALSE END) AS has_bundle_bear
  FROM artifacts
  GROUP BY 1
)

SELECT
  builder_metrics.project_id,
  COALESCE(builder_metrics.transaction_count, 0) AS transaction_count,
  COALESCE(builder_metrics.gas_fees, 0.0) AS gas_fees,
  COALESCE(builder_metrics.active_addresses_count, 0) AS active_addresses_count,
  COALESCE(builder_metrics.active_days, 0) AS active_days,
  builder_metrics.applied_to_round,
  project_eligibility.meets_all_criteria,
  (
    builder_metrics.applied_to_round
    AND project_eligibility.meets_all_criteria
  ) AS is_eligible,
  COALESCE(artifact_flags.has_defillama_adapter, FALSE)
    AS has_defillama_adapter, 
  COALESCE(artifact_flags.has_bundle_bear, FALSE)
    AS has_bundle_bear
FROM builder_metrics
JOIN project_eligibility
  ON builder_metrics.project_id = project_eligibility.project_id
LEFT JOIN artifact_flags
  ON builder_metrics.project_id = artifact_flags.project_id

""")
gm_eligibility.head()

,project_id,transaction_count,gas_fees,active_addresses_count,active_days,applied_to_round,meets_all_criteria,is_eligible,has_defillama_adapter,has_bundle_bear
0,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,2299605,2.02,158928,39,True,True,True,False,False


In [54]:
gm_eligibility.iloc[0]

project_id                klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=
transaction_count                                              2299605
gas_fees                                                          2.02
active_addresses_count                                          158928
active_days                                                         39
applied_to_round                                                  True
meets_all_criteria                                                True
is_eligible                                                       True
has_defillama_adapter                                            False
has_bundle_bear                                                  False
Name: 0, dtype: object

In [57]:
gm_metrics = client.to_pandas("""

WITH base_events AS (
  SELECT
    e.project_id,
    DATE_TRUNC('DAY', e.time) AS bucket_day,
    DATE_TRUNC('MONTH', e.time) AS bucket_month,
    e.event_type,
    e.event_source AS chain,
    e.from_artifact_id,
    e.to_artifact_id,
    e.gas_fee,
    e.transaction_hash,
    COALESCE(users.is_farcaster_user, false) AS is_farcaster_user
  FROM oso.int_superchain_events_by_project AS e
  LEFT OUTER JOIN oso.int_superchain_onchain_user_labels AS users
  ON e.from_artifact_id = users.artifact_id
  WHERE
    /* Currently no 4337-specific logic as these events are 
    already captured in the CONTRACT_INTERNAL_INVOCATION
    events and would need to be de-duped. */ 
    e.event_type IN (
      'CONTRACT_INVOCATION',
      'CONTRACT_INTERNAL_INVOCATION'
    )
    AND e.time >= DATE('2025-01-01') AND e.time < DATE('2025-03-01')
    AND e.project_id = 'klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs='
),

-- Amortized transaction counts
amortized_transaction_count AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'amortized_contract_invocations_monthly' AS metric_name,
    COUNT(DISTINCT transaction_hash) AS amount
  FROM base_events
  WHERE event_type = 'CONTRACT_INTERNAL_INVOCATION'
  GROUP BY 1, 2, 3
),

-- Transaction counts
transaction_count AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'contract_invocations_monthly' AS metric_name,
    COUNT(DISTINCT transaction_hash) AS amount
  FROM base_events
  WHERE event_type = 'CONTRACT_INVOCATION'
  GROUP BY 1, 2, 3
),

-- Gas fees
transaction_gas_fee AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'gas_fees_monthly' AS metric_name,
    SUM(gas_fee) AS amount
  FROM base_events AS e
  WHERE event_type = 'CONTRACT_INVOCATION'
  GROUP BY 1, 2, 3
),

-- Active Farcaster users
monthly_active_farcaster_users AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'active_farcaster_users_monthly' AS metric_name,
    COUNT(DISTINCT from_artifact_id) AS amount
  FROM base_events
  WHERE is_farcaster_user = true
  GROUP BY 1, 2, 3
),

-- Active addresses
monthly_active_addresses AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'active_addresses_monthly' AS metric_name,
    COUNT(DISTINCT from_artifact_id) AS amount
  FROM base_events
  GROUP BY 1, 2, 3
),

union_all_metrics AS (
  SELECT *
  FROM amortized_transaction_count
  UNION ALL
  SELECT *
  FROM transaction_count
  UNION ALL
  SELECT *
  FROM transaction_gas_fee
  UNION ALL
  SELECT *
  FROM monthly_active_farcaster_users
  UNION ALL
  SELECT *
  FROM monthly_active_addresses
)

SELECT
  project_id AS project_id,
  chain AS chain,
  sample_date AS sample_date,
  metric_name AS metric_name,
  amount AS amount
FROM union_all_metrics

""")

gm_metrics['project_name'] = '0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c150e3f72d3dc7ef22dd71a'
gm_metrics['display_name'] = 'GM'
gm_metrics['sample_date'] = gm_metrics['sample_date'].apply(lambda x: x.split(' ')[0])
gm_metrics['measurement_period'] = gm_metrics['sample_date'].apply(lambda x: "Jan 2025" if x == "2025-01-01" else "Feb 2025")

gm_metrics = gm_metrics[[
    'project_id','display_name','project_name','chain',
    'metric_name','sample_date','measurement_period','amount'    
]]

In [70]:
gm_metrics.to_csv("gm_metrics.csv", index=False)

In [41]:
client.to_pandas("""
    select * from oso.projects_v1 where project_name = 'GM'
""")

,project_id,project_source,project_namespace,project_name,display_name,description
0,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,OSS_DIRECTORY,oso,GM,GM,None


In [6]:
df_metrics = client.to_pandas("""
    select
        m.project_id,
        p.display_name,
        pbc.project_name,
        m.chain,
        m.metric_name,
        DATE_FORMAT(m.sample_date, '%Y-%m-%d') AS sample_date,
        DATE_FORMAT(m.sample_date, '%b %Y') AS measurement_period,
        m.amount
    from oso.int_superchain_s7_onchain_metrics_by_project m
    join oso.projects_by_collection_v1 pbc on m.project_id = pbc.project_id
    join oso.projects_v1 p on pbc.project_id = p.project_id
        where pbc.collection_name = '8-1'
        and m.sample_date between date('2025-01-01') and date('2025-02-28')
""")
df_metrics.tail()

,project_id,display_name,project_name,chain,metric_name,sample_date,measurement_period,amount
3446,2d1UftVv4xC+BHialsxoPjC1WJDySvOpnH9o+3sQFYc=,rhinofi,0x85bef37b528ec9d7e1a9cb864e298e3c117071507acd...,MODE,contract_invocations_monthly,2025-01-01,Jan 2025,1333.0
3447,OF1V6dVTQOyDRB5uGdm4yvqv6vkP4TlM22Q/wyrZUZI=,Fabrii2K,0x8f4ac538c2ad2092565a751354d1f11edf4eb3c1ab43...,ZORA,active_addresses_monthly,2025-01-01,Jan 2025,1.0
3448,2d1UftVv4xC+BHialsxoPjC1WJDySvOpnH9o+3sQFYc=,rhinofi,0x85bef37b528ec9d7e1a9cb864e298e3c117071507acd...,MODE,active_addresses_monthly,2025-01-01,Jan 2025,526.0
3449,2d1UftVv4xC+BHialsxoPjC1WJDySvOpnH9o+3sQFYc=,rhinofi,0x85bef37b528ec9d7e1a9cb864e298e3c117071507acd...,MODE,gas_fees_monthly,2025-01-01,Jan 2025,0.000798
3450,dk1gwy0G/dP9SOcyolL+bfgPQJ0ngqXF6ro9qyQdGwo=,Air3,0xf9ebb0464e7162627ab3de5fe03a21c879e932233e14...,ZORA,amortized_contract_invocations_monthly,2025-01-01,Jan 2025,313.2


In [7]:
(df_metrics.query("metric_name == 'average_tvl_monthly'").pivot_table(
    index='display_name',
    columns='sample_date',
    values='amount',
    aggfunc='sum'
) / 1_000_000).sort_values(by='2025-02-01', ascending=False)

sample_date,2025-01-01,2025-02-01
display_name,,
Aerodrome Finance,1245.533253,943.595737
Aave,571.364921,600.653132
Uniswap on Superchain (Oku),698.287355,480.403061
Morpho,459.004645,479.616796
Moonwell,178.625471,144.383072
Stargate Finance,76.937209,143.410985
Bedrock on Optimism Superchain,142.024507,135.238943
Compound Finance,118.35966,110.635796
Extra Finance,140.948681,93.446806


In [10]:
DIR = "../../../../../../../../GitHub/Retro-Funding/"
#eligibility.to_csv(DIR + "eval-algos/S7/data/M1/onchain/onchain_builders.csv", index=False)
#df_metrics.to_csv(DIR + "eval-algos/S7/data/M1/onchain/onchain_metrics_by_project.csv", index=False)

In [34]:
ae = client.to_pandas("""

select
    DATE_FORMAT(time, '%b %Y') AS measurement_period,
    event_type,
    SUM(gas_fee)
from int_superchain_events_by_project
where 
    project_id = 'rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo='
group by 1,2

""")

In [37]:
ae[ae['measurement_period'] == 'Jan 2025']

,measurement_period,event_type,_col2
2,Jan 2025,CONTRACT_INVOCATION_VIA_USEROP,861803.09
5,Jan 2025,CONTRACT_INVOCATION,264.87
17,Jan 2025,CONTRACT_INTERNAL_INVOCATION,1985.13


In [23]:
pd.options.display.float_format = '{:.2f}'.format

In [24]:
df_metrics_old = pd.read_csv(DIR + "eval-algos/S7/data/M1/onchain/onchain_metrics_by_project.csv")
df_metrics_old['amount'] = df_metrics_old['amount'].apply(lambda x: round(x,5))

In [33]:
(
    df_metrics_old[df_metrics_old['display_name'] == 'Aerodrome Finance']
    [['project_id', 'chain', 'metric_name', 'sample_date', 'amount']]
    #.query('metric_name == "gas_fees_monthly"')
    .sort_values(by='amount')
)

,project_id,chain,metric_name,sample_date,amount
3006,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,gas_fees_monthly,2025-02-01,182.69
871,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,gas_fees_monthly,2025-01-01,265.01
883,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,active_farcaster_users_monthly,2025-01-01,37607.00
3010,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,active_farcaster_users_monthly,2025-02-01,40325.00
2998,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,active_addresses_monthly,2025-02-01,370798.00
859,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,active_addresses_monthly,2025-01-01,502077.00
840,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,amortized_contract_invocations_monthly,2025-01-01,43258239.60
3029,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,amortized_contract_invocations_monthly,2025-02-01,45640742.00
3024,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,contract_invocations_monthly,2025-02-01,71373056.00
864,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,BASE,contract_invocations_monthly,2025-01-01,72736610.00


In [32]:
(
    df_metrics[df_metrics['display_name'] == 'Aerodrome Finance']
    [['chain', 'metric_name', 'sample_date', 'amount']]
    #.query('metric_name == "gas_fees_monthly"')
    .sort_values(by='amount')
)

,chain,metric_name,sample_date,amount
1279,BASE,gas_fees_monthly,2025-02-01,152.94
2645,BASE,gas_fees_monthly,2025-01-01,265.01
1288,BASE,active_farcaster_users_monthly,2025-02-01,28188.00
2617,BASE,active_farcaster_users_monthly,2025-01-01,37607.00
1292,BASE,active_addresses_monthly,2025-02-01,303192.00
2626,BASE,active_addresses_monthly,2025-01-01,502077.00
1309,BASE,amortized_contract_invocations_monthly,2025-02-01,34268919.60
2616,BASE,amortized_contract_invocations_monthly,2025-01-01,43258239.60
1298,BASE,contract_invocations_monthly,2025-02-01,53863558.00
2636,BASE,contract_invocations_monthly,2025-01-01,72736610.00


,project_id,display_name,project_name,chain,metric_name,sample_date,measurement_period,amount
0,xOZNIZHXWrA+JWBmcsHybyHDy0WZy/gOyw1VaqAupG0=,Kelp,0xff1edf67227651293a9cff4dce3577c6ca52bf8c7c6f...,BASE,average_tvl_monthly,2025-01-01,Jan 2025,1.788566e+07
1,xOZNIZHXWrA+JWBmcsHybyHDy0WZy/gOyw1VaqAupG0=,Kelp,0xff1edf67227651293a9cff4dce3577c6ca52bf8c7c6f...,BASE,amortized_contract_invocations_monthly,2025-01-01,Jan 2025,5.001400e+03
2,xOZNIZHXWrA+JWBmcsHybyHDy0WZy/gOyw1VaqAupG0=,Kelp,0xff1edf67227651293a9cff4dce3577c6ca52bf8c7c6f...,BASE,gas_fees_monthly,2025-01-01,Jan 2025,6.292967e-06
3,Jr4krZ4lp1dMiTRDwmF2R9Hw4mF3rsukcBCM4pnF/fc=,Uniswap on Superchain (Oku),0x9d9162fac637991d2a56da8fbe03dba24c87eb82d47c...,LISK,contract_invocations_monthly,2025-01-01,Jan 2025,5.089071e+06
4,b0ma4f7o65FUNGRC2R4eb8Fo7DB6imYfqrM3S0JQ4tc=,Gains Network (gTrade),0x70ba3cb2e433f1cd9ace8aea84c46d167ba5db12293c...,BASE,contract_invocations_monthly,2025-01-01,Jan 2025,1.437760e+05
...,...,...,...,...,...,...,...,...
3493,D/phNjIkMkuESYi0XbICo299wHOagQbo2WSAyODxDp0=,Stargate Finance,0x62e37e96aa6e1cbfb6bd24b97c4b8f1e12cc3fe35d53...,BASE,contract_invocations_monthly,2025-02-01,Feb 2025,4.029100e+04
3494,LLK6remJzmNWYHJv5LsqViPMz+6FR4LML+t1E9V62WQ=,Records.xyz,0xdd5998cc29bcd164fe277beb1e64f1fb28390886bc73...,OPTIMISM,contract_invocations_monthly,2025-02-01,Feb 2025,3.900000e+01
3495,sw8ehEEk8y0FK2GSDVnFLz7M9erX6RSQdSAgGDmiwEs=,LI.FI,0x517eaa9c56951de89261f2d7830ea49aae92f2a90310...,SONEIUM,gas_fees_monthly,2025-02-01,Feb 2025,9.224369e-05
3496,7Iv5H47guOZI+q3EgByroFWR0Dw7uoImhWPboCDC0FQ=,far.quest - Farcaster Super App,0x182f5a5c04ccdc4739ee9f362af72d7a0f55d6b410cc...,BASE,active_farcaster_users_monthly,2025-02-01,Feb 2025,8.530000e+02


In [ ]:
df_rewards = pd.read_csv(
    "../Retro-Funding/eval-algos/S7/data/M1/onchain/onchain_builders_m1_goldilocks_rewards.csv"
)
df_eligibility_dump = eligibility.merge(
    df_rewards[['project_name', 'op_reward']],
    left_on='project_name',
    right_on='project_name',
    how='outer'
)
df_eligibility_dump['is_eligible'] = df_eligibility_dump['is_eligible'].fillna(False)
df_eligibility_dump['round_id'] = '8'
cols = [
    'project_id', 'project_name', 'display_name', 
    'round_id',  'is_eligible', 'op_reward',
    'transaction_count', 'active_days', 'active_addresses_count',
    'gas_fees', 'has_defillama_adapter', 'has_bundle_bear'
]
df_eligibility_dump = df_eligibility_dump[cols]

json_str = df_eligibility_dump.to_json(orient='records', indent=2)
clean_json_str = json_str.replace('\\/', '/')
with open('m1_onchain_builders_eligibility.json', 'w') as file:
    file.write(clean_json_str)